In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt

import json

import kagglehub


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import ParameterGrid

from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score

In [ ]:
# Download latest version
path = kagglehub.dataset_download("rabieelkharoua/alzheimers-disease-dataset")
files = os.listdir(path)
print("Content of", files)

csv_file = files[0]
csv_path = os.path.join(path, csv_file)

# Load DataFrame
df = pd.read_csv(csv_path)

Content of ['alzheimers_disease_data.csv']


In [8]:
# Preprocessing

# Remove unnecessary columns
df = df.drop(columns=["PatientID", "DoctorInCharge"], errors="ignore")

# Splitting the data into features (X) and the target variable (Y)
X = df.drop(columns=["Diagnosis"])
Y = df["Diagnosis"]


# Label encoding
def change_labels(X):
    custom_labels = {
        "Gender": ["Male", "Female"],
        "Ethnicity": ["Caucasian", "African American", "Asian", "Other"],
        "EducationLevel": ["None", "High School", "Bachelor's", "Higher"],
        "Smoking": ["No", "Yes"],
        "FamilyHistoryAlzheimers": ["No", "Yes"],
        "CardiovascularDisease": ["No", "Yes"],
        "Diabetes": ["No", "Yes"],
        "Depression": ["No", "Yes"],
        "HeadInjury": ["No", "Yes"],
        "Hypertension": ["No", "Yes"],
        "MemoryComplaints": ["No", "Yes"],
        "BehavioralProblems": ["No", "Yes"],
        "Confusion": ["No", "Yes"],
        "Disorientation": ["No", "Yes"],
        "PersonalityChanges": ["No", "Yes"],
        "DifficultyCompletingTasks": ["No", "Yes"],
        "Forgetfulness": ["No", "Yes"],
    }

    for column, labels in custom_labels.items():
        if column in X.columns:
            # Create a mapping dictionary from numeric values to custom labels
            label_mapping = {i: label for i, label in enumerate(labels)}
            # Replace values in the column using the mapping
            X[column] = X[column].replace(label_mapping)
    return X


# Assigning custom labels
X = change_labels(X)

# Splitting the dataset into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y
)

In [9]:
def objective(trial):
    # Przestrzeń wyszukiwania hiperparametrów
    max_bins = trial.suggest_categorical("max_bins", [128, 256, 512])
    max_interaction_bins = trial.suggest_categorical(
        "max_interaction_bins", [32, 64, 128]
    )
    interactions = trial.suggest_categorical("interactions", [0, 10, 50])
    learning_rate = trial.suggest_float("learning_rate", 0.01, 1, log=True)
    min_samples_leaf = trial.suggest_categorical("min_samples_leaf", [2, 10, 20])
    max_leaves = trial.suggest_categorical("max_leaves", [3, 5, 10])

    # Model EBM z sugerowanymi hiperparametrami
    ebm = ExplainableBoostingClassifier(
        max_bins=max_bins,
        max_interaction_bins=max_interaction_bins,
        interactions=interactions,
        learning_rate=learning_rate,
        min_samples_leaf=min_samples_leaf,
        max_leaves=max_leaves,
        random_state=42,
        n_jobs=-1,
    )

    # Dopasowanie modelu na zestawie treningowym
    ebm.fit(X_train, Y_train)

    # Przewidywania na zbiorze walidacyjnym
    Y_pred = ebm.predict(X_test)

    # Obliczanie wyniku recall
    score = recall_score(Y_test, Y_pred, average="binary")
    return score


# Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best hyperparameters:", study.best_params)
print("Best recall score:", study.best_value)

[I 2025-01-28 10:29:12,759] A new study created in memory with name: no-name-a0c906fa-52b2-4cae-b632-019531be7cd2
[I 2025-01-28 10:29:26,107] Trial 0 finished with value: 0.8881578947368421 and parameters: {'max_bins': 256, 'max_interaction_bins': 128, 'interactions': 50, 'learning_rate': 0.749577512667121, 'min_samples_leaf': 10, 'max_leaves': 3}. Best is trial 0 with value: 0.8881578947368421.
[I 2025-01-28 10:29:37,692] Trial 1 finished with value: 0.9210526315789473 and parameters: {'max_bins': 512, 'max_interaction_bins': 128, 'interactions': 10, 'learning_rate': 0.026971177558532598, 'min_samples_leaf': 20, 'max_leaves': 5}. Best is trial 1 with value: 0.9210526315789473.
[I 2025-01-28 10:29:41,401] Trial 2 finished with value: 0.881578947368421 and parameters: {'max_bins': 256, 'max_interaction_bins': 128, 'interactions': 50, 'learning_rate': 0.6260932989680296, 'min_samples_leaf': 10, 'max_leaves': 10}. Best is trial 1 with value: 0.9210526315789473.
[I 2025-01-28 10:29:44,387]

Best hyperparameters: {'max_bins': 512, 'max_interaction_bins': 64, 'interactions': 10, 'learning_rate': 0.02037139302656449, 'min_samples_leaf': 2, 'max_leaves': 5}
Best recall score: 0.9276315789473685


In [10]:
from functions.save_load_model import save_model, load_model

# save_model(study, "ebm_study.json")

Model has been saved as models/trained/ebm_study.json.joblib


In [11]:
load_ebm = load_model("ebm_study.json")

In [12]:
# Fitting the Explainable Boosting Classifier model
ebm_model = ExplainableBoostingClassifier(
    random_state=42,
    max_bins=512,
    max_interaction_bins=128,
    interactions=10,
    learning_rate=0.010226651476661184,
    min_samples_leaf=2,
    max_leaves=3,
)
ebm_model.fit(X_train, Y_train)

# Generating predictions and probabilities on the test set
Y_pred = ebm_model.predict(X_test)
Y_pred_proba = ebm_model.predict_proba(X_test)[:, 1]

# Model evaluation
print("EBM - Classification Report:")
print(classification_report(Y_test, Y_pred))
print(f"EBM - AUC ROC: {roc_auc_score(Y_test, Y_pred_proba):.4f}")

EBM - Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.95      0.95       278
           1       0.90      0.93      0.92       152

    accuracy                           0.94       430
   macro avg       0.93      0.94      0.93       430
weighted avg       0.94      0.94      0.94       430

EBM - AUC ROC: 0.9452


In [ ]:
ebm_local = ebm_model.explain_local(X_test[:10], Y_test[:10])
show(ebm_local)